In [ ]:
# %% [markdown]
# # RDT 实验结果分析与可视化 (Analysis & Visualization)
#
# 本 Notebook 用于加载已完成的 RDT 实验结果，并利用 `src/utils.py` 中的函数进行可视化分析。

# %%
import os
import sys
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import glob # 用于查找文件

# %% [markdown]
# ## 1. 配置与加载 (Configuration & Loading)
#
# **重要**: 设置 `experiment_dir_to_load` 指向您想要分析的具体实验结果目录。

# %%
# --- 用户需要配置 ---
# 例如: 'results/weatherHistory_DLinear_PatchTST_20231027_153000'
experiment_dir_to_load = "results/weatherHistory0501"
# --------------------

# 检查目录是否存在
if not os.path.isdir(experiment_dir_to_load):
    raise FileNotFoundError(f"指定的实验目录不存在: {experiment_dir_to_load}")

# 定义子目录路径
metrics_dir = os.path.join(experiment_dir_to_load, 'metrics')
models_dir = os.path.join(experiment_dir_to_load, 'models')
plots_dir = os.path.join(experiment_dir_to_load, 'plots') # 可能用于查找已有图或保存新图

# 将 src 目录添加到 Python 路径，以便导入自定义模块
# (假设 notebook 在项目根目录，src 在其下)
project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# 导入必要的自定义模块
try:
    from src import utils
    from src import config as current_config # 加载当前配置以获取模型结构等信息
    from src import models
    from src.data_handler import load_and_preprocess_data # 如果需要重新预测
    print("自定义模块导入成功。")
    print(f"当前配置中的教师模型: {current_config.TEACHER_MODEL_NAME}, 学生模型: {current_config.STUDENT_MODEL_NAME}")
except ImportError as e:
    print(f"导入自定义模块失败: {e}")
    print("请确保 Notebook 文件位于项目根目录，或者调整 sys.path 设置。")
    # 或者您可以将 src 目录下的文件复制到 notebook 所在的目录（不推荐）

# %% [markdown]
# ### 1.1 加载指标数据 (Load Metrics Data)

# %%
# 查找主要的运行结果 CSV 文件 (通常只有一个)
main_results_files = glob.glob(os.path.join(metrics_dir, "all_runs_summary_*.csv"))
if not main_results_files:
    print(f"错误: 在 {metrics_dir} 中找不到 'all_runs_summary_*.csv' 文件。")
    results_df = pd.DataFrame() # 创建空 DataFrame 以避免后续错误
else:
    main_results_path = main_results_files[0]
    print(f"加载主结果文件: {main_results_path}")
    results_df = pd.read_csv(main_results_path)
    display(results_df.head())

# 查找稳定性总结文件 (如果存在)
stability_files = glob.glob(os.path.join(metrics_dir, "stability_summary_*.csv"))
if stability_files:
    stability_path = stability_files[0]
    print(f"加载稳定性总结文件: {stability_path}")
    stability_df = pd.read_csv(stability_path, index_col=0)
    display(stability_df)
else:
    print("未找到稳定性总结文件 (可能是单次运行)。")
    stability_df = None

# 加载鲁棒性结果文件
robustness_files = glob.glob(os.path.join(metrics_dir, "*_robustness.csv"))
robustness_data = {}
if robustness_files:
    print("\n加载鲁棒性结果文件:")
    for f_path in robustness_files:
        filename = os.path.basename(f_path)
        # 从文件名中提取模型名称 (去除 _runX_robustness.csv 部分)
        try:
             # 假设文件名格式如 Teacher_run0_robustness.csv 或 Student_RDT_run0_robustness.csv
             model_name_parts = filename.split('_run')[0] # 获取 run 之前的部分
             # 如果原始脚本保存的名字包含run ID，上面这样提取可能不准确，更好的方式是从 results_df 获取模型前缀
             # 简单起见，先用上面的方法，或者直接打印文件名让用户确认
             model_key = model_name_parts # 使用提取的名称作为字典的键
             print(f"- {filename} -> 键: {model_key}")
             df_robust = pd.read_csv(f_path, index_col=0)
             robustness_data[model_key] = df_robust
        except Exception as e:
             print(f"  处理文件 {filename} 时出错: {e}")
    if robustness_data:
        display(list(robustness_data.values())[0].head()) # 显示第一个加载的 DataFrame 示例
else:
    print("未找到鲁棒性结果文件。")


# %% [markdown]
# ### 1.2 (可选) 加载模型 (Optional: Load Models)
#
# 如果您需要使用加载的模型进行新的预测或分析，可以取消注释并运行以下单元格。
# **注意:** 这需要当前的 `src/config.py` 与训练时的模型配置兼容。

# %%
# load_models_flag = False # 设置为 True 以加载模型

# if load_models_flag and not results_df.empty:
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     loaded_models = {}
#     model_path_cols = [col for col in results_df.columns if col.endswith('_model_path')]

#     # 通常我们只需要加载每个模型类型的最佳/最后一次运行的模型
#     # 这里简化为加载第一次运行 (run_id=0) 的模型路径
#     first_run_paths = results_df[results_df['run_id'] == 0][model_path_cols].iloc[0].to_dict()

#     print("\n尝试加载模型 (来自 Run 0):")
#     for model_key_col, model_path in first_run_paths.items():
#         if pd.isna(model_path) or not os.path.exists(model_path):
#             print(f"- 无法加载 {model_key_col}: 路径无效或不存在 ({model_path})")
#             continue

#         # 从列名推断模型类型 (例如 'teacher_model_path' -> 'teacher')
#         model_type = model_key_col.split('_model_path')[0]

#         try:
#             print(f"- 加载 {model_type} 从 {model_path}...")
#             if model_type == 'teacher':
#                 model_instance = models.get_teacher_model(current_config)
#             elif model_type == 'student_task_only' or model_type == 'student_rdt' or model_type == 'student_follower':
#                 # 假设所有学生模型使用相同的架构
#                 model_instance = models.get_student_model(current_config)
#             else:
#                 print(f"  未知的模型类型: {model_type}, 跳过。")
#                 continue

#             loaded_model = utils.load_model(model_instance, model_path, device)
#             if loaded_model:
#                 loaded_models[model_type] = loaded_model
#                 print(f"  成功加载 {model_type}")
#             else:
#                 print(f"  加载失败 {model_type}")

#         except Exception as e:
#             print(f"  加载 {model_type} 时出错: {e}")

#     print("\n可用已加载模型:", list(loaded_models.keys()))

# else:
#      print("\n未加载模型 (load_models_flag=False 或结果文件为空)。")


# %% [markdown]
# ## 2. 可视化分析 (Visualization & Analysis)

# %% [markdown]
# ### 2.1 指标对比 (Metric Comparison)
#
# 比较不同模型在测试集上的平均性能指标 (如果进行了多次运行，则基于平均值)。

# %%
if not results_df.empty:
    # 提取每个模型类型的平均指标
    metric_cols = [m for m in current_config.METRICS] # e.g., ['mae', 'mse']
    model_prefixes = ["Teacher", "Student_TaskOnly", "Student_RDT", "Student_Follower"]
    avg_metrics = {}

    # 计算所有运行的平均指标
    grouped = results_df.mean(numeric_only=True) # 计算所有数值列的均值

    for prefix in model_prefixes:
        model_metrics = {}
        for metric in metric_cols:
            col_name = f"{prefix}_{metric}"
            if col_name in grouped:
                model_metrics[metric] = grouped[col_name]
        if model_metrics: # 只有当该模型至少有一个指标时才添加
            avg_metrics[prefix] = model_metrics

    if avg_metrics:
        print("用于绘图的平均指标:")
        print(pd.DataFrame(avg_metrics).T)

        # 定义保存路径 (在原始实验的 plots 目录下)
        save_path = os.path.join(plots_dir, "notebook_avg_metric_comparison.png")

        utils.plot_metric_comparison(
            metrics_dict=avg_metrics,
            title=f"Average Test Set Metric Comparison ({os.path.basename(experiment_dir_to_load)})",
            save_path=save_path
        )
        print(f"指标对比图已保存到: {save_path}")
        # 在 notebook 中显示图像
        if os.path.exists(save_path):
            from IPython.display import Image
            display(Image(filename=save_path))
    else:
        print("未找到足够的平均指标数据进行绘图。")
else:
    print("结果 DataFrame 为空，无法绘制指标对比图。")

# %% [markdown]
# ### 2.2 稳定性对比 (Stability Comparison)
#
# 如果实验运行了多次 (`STABILITY_RUNS > 1`)，使用箱线图或小提琴图比较不同模型指标的分布。

# %%
if not results_df.empty and 'run_id' in results_df.columns and results_df['run_id'].max() > 0:
    print("\n生成稳定性对比图...")
    for metric in current_config.METRICS:
        save_path = os.path.join(plots_dir, f"notebook_stability_comparison_{metric}.png")
        try:
            utils.plot_stability_comparison(
                results_df=results_df,
                metric_to_plot=metric,
                title=f"Stability Comparison ({metric.upper()}) ({os.path.basename(experiment_dir_to_load)})",
                save_path=save_path,
                plot_type='box' # 或 'violin'
            )
            print(f"稳定性对比图 ({metric}) 已保存到: {save_path}")
            if os.path.exists(save_path):
                from IPython.display import Image
                display(Image(filename=save_path))
        except Exception as e:
            print(f"绘制稳定性图 ({metric}) 时出错: {e}")
else:
    print("\n未进行多次运行或结果文件为空，跳过稳定性对比图。")

# %% [markdown]
# ### 2.3 鲁棒性对比 (Robustness Comparison)
#
# 如果进行了鲁棒性测试，绘制模型性能随噪声水平变化的曲线。

# %%
if robustness_data:
     print("\n生成鲁棒性对比图...")
     # 需要调整 robustness_data 的键以匹配模型名称 (如果加载时键不准确)
     # 例如，假设 results_df 包含正确的模型前缀
     model_name_map = { # 可能需要手动调整或从 results_df 推断
         "Teacher": "Teacher",
         "Student_TaskOnly": "Student_TaskOnly",
         "Student_RDT": "Student_RDT",
         "Student_Follower": "Student_Follower"
     }
     # 过滤掉 robustness_data 中不存在于 map 中的键
     filtered_robustness_data = {model_name_map.get(k): v for k, v in robustness_data.items() if model_name_map.get(k)}

     if filtered_robustness_data:
         for metric in current_config.METRICS:
             save_path = os.path.join(plots_dir, f"notebook_robustness_comparison_{metric}.png")
             try:
                 utils.plot_robustness_comparison(
                     robustness_results_dict=filtered_robustness_data, # 使用过滤/映射后的字典
                     metric_name=metric,
                     title=f"Robustness Comparison ({metric.upper()}) ({os.path.basename(experiment_dir_to_load)})",
                     save_path=save_path
                 )
                 print(f"鲁棒性对比图 ({metric}) 已保存到: {save_path}")
                 if os.path.exists(save_path):
                     from IPython.display import Image
                     display(Image(filename=save_path))
             except Exception as e:
                 print(f"绘制鲁棒性图 ({metric}) 时出错: {e}")
     else:
          print("无法将加载的鲁棒性数据键映射到已知模型名称，跳过绘图。请检查 robustness_data 字典。")
else:
    print("\n未找到鲁棒性数据，跳过鲁棒性对比图。")


# %% [markdown]
# ### 2.4 (可选) 重新生成预测对比图 (Optional: Regenerate Prediction Plots)
#
# 如果您加载了模型和数据，可以重新生成预测对比图。

# %%
# regenerate_predictions = False # 设置为 True 以运行

# if regenerate_predictions and loaded_models and 'test_loader' in locals(): # 确保模型和数据已加载
#     print("\n重新生成预测对比图...")
#     device = next(iter(loaded_models.values())).device # 获取模型的设备
#     all_preds_dict = {}
#     true_values_original = None

#     # 使用 DataLoader 获取一批或全部真实值和预测值
#     # 注意: 这可能与原始评估略有不同，取决于 DataLoader 状态
#     # 简单起见，我们只用 evaluator.predict 来获取所有预测
#     print("正在重新获取预测值...")
#     for name, model in loaded_models.items():
#         # 需要 test_loader 和 scaler
#         # 假设我们能从当前配置重新加载它们
#         try:
#              if 'train_loader' not in locals(): # 避免重复加载
#                  _, _, test_loader, scaler = load_and_preprocess_data(current_config)

#              trues_scaled, preds_scaled = evaluator.predict(model, test_loader, device) # 使用导入的 evaluator

#              # 逆变换
#              n_samples, horizon, n_features = preds_scaled.shape
#              pred_reshaped = preds_scaled.view(-1, n_features).cpu().numpy()
#              true_reshaped = trues_scaled.view(-1, n_features).cpu().numpy()
#              preds_original = scaler.inverse_transform(pred_reshaped).reshape(n_samples, horizon, n_features)
#              trues_original_temp = scaler.inverse_transform(true_reshaped).reshape(n_samples, horizon, n_features)

#              all_preds_dict[name] = preds_original
#              if true_values_original is None:
#                  true_values_original = trues_original_temp
#              print(f"- 获取到 {name} 的预测")

#         except Exception as e:
#              print(f"获取 {name} 预测时出错: {e}")

#     if true_values_original is not None and all_preds_dict:
#         save_path = os.path.join(plots_dir, "notebook_comparison_predictions.png")
#         utils.plot_comparison_predictions(
#             true_values=true_values_original,
#             predictions_dict=all_preds_dict,
#             title=f"Prediction Comparison (Regenerated) ({os.path.basename(experiment_dir_to_load)})",
#             save_path=save_path,
#             series_idx=0 # 选择要绘制的序列
#         )
#         print(f"预测对比图已保存到: {save_path}")
#         if os.path.exists(save_path):
#             from IPython.display import Image
#             display(Image(filename=save_path))
#     else:
#          print("未能获取足够的预测数据来绘图。")

# else:
#     print("\n未重新生成预测图 (regenerate_predictions=False 或缺少模型/数据加载器)。")


# %% [markdown]
# ## 3. 自定义分析 (Custom Analysis)
#
# 您可以在此添加任何其他基于加载数据的分析或可视化代码。例如，查看特定运行的详细指标、比较不同 alpha 调度策略等。

# %%
# 示例：显示特定模型的平均指标
# model_to_show = 'Student_RDT'
# if model_to_show in avg_metrics:
#     print(f"\n{model_to_show} 的平均指标:")
#     print(pd.Series(avg_metrics[model_to_show]))

# %%
# 示例：查看结果 DataFrame 的详细信息
# if not results_df.empty:
#     print("\n结果 DataFrame 信息:")
#     results_df.info()
#     print("\n结果 DataFrame 描述统计:")
#     display(results_df.describe())


